In [1]:
"""
Find modules based on correlation between expression and signatures

1) select genes with significant correlations
1-1) all significant genes
1-2) subset of genes (DNA metabolic etc)
2) cluster genes
input: (output files from comp_corr_sigma.ipynb)
corr_count_file "results/corr_expr_sigma_count.tsv"
pv_count_file "results/corr_pv_expr_sigma_count.tsv"

output: 
cluste id file "results/all_cluster_id.tsv"
"""

In [6]:
import pandas as pd
import numpy as np

import scipy 
import scipy.stats as ss
import scipy.cluster.hierarchy as sch
from sklearn.cluster import KMeans, MiniBatchKMeans
import sklearn.cluster as cluster
from sklearn import metrics
import statsmodels.stats.multitest as ssm

In [2]:
# input files
corr_count_file = "results/corr_expr_sigma_count.tsv"
pv_count_file = "results/corr_pv_expr_sigma_count.tsv"
go_file = "data/c5.bp.v6.1.symbols.gmt"

In [4]:
# read correlation file
pv_count = pd.read_csv(pv_count_file, sep="\t", index_col=0)
corr_count = pd.read_csv(corr_count_file, sep="\t", index_col=0)

# take care of duplicates
def rename_dp(df):
    df.index = df.index + df.groupby(level=0).cumcount().astype(str).replace('1', '_1').replace('2', '_2').replace('0','')

rename_dp(corr_count)
rename_dp(pv_count)

sigs = ["1D", "2C", "2D", "3C", "3D", "5D", "8C", "8D", "13C", "13D"]

corr_count = corr_count[sigs]
pv_count = pv_count[sigs]

In [7]:
# select genes based on pvalues and correlation
# threshold for correlation coefs and p-values
corr_th = 0.3
fdr_th = 0.005

sig_corr_dic = {}
sig_corr_genes = set([])

for sig in sigs:
    print(sig)
    rej, pvc, a, b = ssm.multipletests(pv_count[sig], fdr_th, method="fdr_bh")
    sig_corr = pv_count.index[rej & ((corr_count[sig] >= corr_th)|(corr_count[sig] <= -corr_th))]
    print(len(sig_corr))
    if len(sig_corr) == 0:
        continue
    sig_corr_dic[sig] = sig_corr
    sig_corr_genes = sig_corr_genes.union(sig_corr)

1D
6
2C
157
2D
7
3C
2311
3D
3259
5D
93
8C
1343
8D
14
13C
760
13D
838


In [9]:
# GO genes
lines = [l.split() for l in open(go_file).readlines()]
go_dic = {}
for l in lines:
    go_dic[l[0]] = l[2:]
    
# GO_meta_genes
dna_meta_genes = list(filter(lambda l: l[0] == "GO_DNA_METABOLIC_PROCESS", lines))[0][2:]

In [10]:
# find correlation table only for genes of interst
genes_of_interest = sig_corr_genes
def create_corr_dic(corr_df, genes_of_interest):
    corr_subset = corr_df[corr_df.index.isin(genes_of_interest)]
    return corr_subset

corr_count_selected = create_corr_dic(corr_count, genes_of_interest)
corr_count_meta = create_corr_dic(corr_count_selected, dna_meta_genes)

In [11]:
# Options for clustering
GO="all"
if GO == "META":
    matrix_to_cluster = corr_count_meta
    prefix = "meta_" 
    # sig_go_dic = sig_meta_go_dic
    subset_genes = dna_meta_genes 
    denom=10
    step=2
elif GO == "all":
    matrix_to_cluster = corr_count_selected
    prefix = "all_" 
    # sig_go_dic = go_dic
    subset_genes =  list(corr_count.index)
    denom=10
    step=5

In [12]:
# Clustering genes:
# Step 1: k-means clustering
labels_all = []
for ri in range(100):
    if ri%10 == 0:
        print(ri)
    clusterer_ri = MiniBatchKMeans(n_clusters=(int(ri/denom)+1)*step, random_state=ri)
    labels = clusterer_ri.fit_predict(matrix_to_cluster)
    labels_all.append(labels)

labels_df = pd.DataFrame(labels_all, columns=matrix_to_cluster.index)

0
10
20
30
40
50
60
70
80
90


In [13]:
# Step 2: find consensus score for each 
def count_consensus(df, label):
    return (df == label).astype(int).sum(1)

consensus_df = pd.DataFrame()
labels_T = labels_df.T
for i in range(labels_df.shape[1]):
    if (i%1000==0): # print progress
        print(i)
    consensus_df[i] = count_consensus(labels_T, labels_T.iloc[i,:])

corr_consensus_summary_file = "../results/"+prefix+"corr_consensus.tsv"
consensus_df.columns=consensus_df.index

0
1000
2000
3000


In [ ]:
# Step 3: hierarchical clustering
consensus_mat = consensus_df.as_matrix()
link_matrix = sch.linkage(consensus_mat, method="complete", metric="cosine") 
cluster_id_df = pd.DataFrame(index=matrix_to_cluster.index)
nrange = range(7, 8)
for n_clusters in nrange:
    print(n_clusters)
    th = [x[2] for x in link_matrix][-n_clusters]
    labels = sch.fcluster(link_matrix, th, criterion='distance')
    cluster_id_df["k="+str(n_clusters)] = labels
    

In [21]:
# write the results
cluster_id_df.to_csv("../results/"+prefix+"cluster_id.tsv", sep="\t")